In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType

df = spark.read.table("workspace.bronze.erp_accessory_category")

**Transform Data**

In [0]:
for fields in df.schema.fields:
    if isinstance(fields.dataType,StringType):
        df = df.withColumn(fields.name,F.trim(F.col(fields.name)))


In [0]:
%sql
select * from workspace.bronze.erp_accessory_category


Normalisation

In [0]:
df = df.withColumn("MAINTENANCE",
                   F.when(F.upper(F.col("MAINTENANCE")) == "YES", F.lit(True))
                   .when(F.upper(F.col("MAINTENANCE")) == "NO",F.lit(False))
                   .otherwise(None))

**Rename Column**

In [0]:
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)
     

**Write to silver table**

In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.silver.erp_accessory_category")
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_accessory_category")